In [30]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: 'SOS', 1: 'EOS'}
        self.n_words = 2 # 'SOS', 'EOS'

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else :
            self.word2count[word] += 1


In [93]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r'([.!?])', r" \1", s)
    s = re.sub(r'[^a-zA-Z.!?]+', r" ", s)
    return s

In [106]:
def readLangs(lang1, lang2, reverse=False):
    print('Reading Lines...')

    # 줄별로 구분해서 읽기
    %cd /content/drive/MyDrive/TIL/data
    lines = open('%s-%s.txt' % (lang1,lang2), encoding='utf-8').\
        read().strip().split('\n')

    # pairs로 줄별로 분리하고 normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse 고려
    if reverse :
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else :
        input_lang = Lang(lang1)
        output_lang =Lang(lang2)
    
    return input_lang, output_lang, pairs

In [107]:
MAX_LENGTH = 10
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [110]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print('Read %s sentence pairs' % len(pairs))
    pairs = filterPairs(pairs)
    print('Trimmed to %s sentence pairs' % len(pairs))
    print('Counted words:')
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading Lines...
/content/drive/MyDrive/TIL/data
Read 326480 sentence pairs
Trimmed to 22389 sentence pairs
Counted words:
fra 6967
eng 4986
['je ne fais qu une suggestion .', 'i m just making a suggestion .']


In [113]:
input_lang.index2word

{0: 'SOS',
 1: 'EOS',
 2: 'j',
 3: 'en',
 4: 'perds',
 5: 'mes',
 6: 'mots',
 7: '.',
 8: 'les',
 9: 'me',
 10: 'manquent',
 11: 'je',
 12: 'ne',
 13: 'trouve',
 14: 'pas',
 15: 'tu',
 16: 'es',
 17: 'meilleure',
 18: 'forme',
 19: 'que',
 20: 'moi',
 21: 'sur',
 22: 'mon',
 23: 'chemin',
 24: 'vous',
 25: 'etes',
 26: 'vais',
 27: 'y',
 28: 'aller',
 29: 'tellement',
 30: 'impatiente',
 31: 'avec',
 32: 'il',
 33: 'donne',
 34: 'des',
 35: 'coups',
 36: 'de',
 37: 'pied',
 38: '!',
 39: 'suis',
 40: 'si',
 41: 'gros',
 42: 'le',
 43: 'descendre',
 44: 'fais',
 45: 'parler',
 46: 'elle',
 47: 'demande',
 48: 'comment',
 49: 'c',
 50: 'est',
 51: 'possible',
 52: 't',
 53: 'vraiment',
 54: 'bete',
 55: 'curieux',
 56: 'un',
 57: 'grand',
 58: 'voyageur',
 59: 'n',
 60: 'assez',
 61: 'rapide',
 62: 'rapides',
 63: 'ange',
 64: 'on',
 65: 'se',
 66: 'voit',
 67: 'demain',
 68: '?',
 69: 'tres',
 70: 'sexy',
 71: 'fatigue',
 72: 'fatiguee',
 73: 'meurs',
 74: 'faim',
 75: 'creve',
 76: 'ma

In [114]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [115]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)


    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device = device)